# Capstone: Visualizations

**San Francisco Police Department Incident Reports**

This notebook supports the visualizations for the final report of this Capstone Project. Please reference 
the [README.md](https://github.com/fazeelgm/UCB_ML_AI_Capstone/blob/main/README.md) 
for details.

## Imports & Utilities

### Imports

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.image as mpimg


# Import utilities
# import pathlib
import time

# Export dataFrame's as images
import dataframe_image as dfi

# import project utils
import sys
sys.path.append('../src')

import data_utils
from data_utils import Config

import graph_utils

from sklearn.preprocessing import StandardScaler, LabelEncoder

In [6]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.INFO)

### Utility Functions

In [8]:
def time_secs_to_msg(lapse_time_secs, mins_label='m', secs_label='s'):
    if lapse_time_secs <= 60:
        return f'{lapse_time_secs%60:.2f}{secs_label}'
    else:
        return f'{lapse_time_secs//60:,.0f}{mins_label} {lapse_time_secs%60:.2f}{secs_label}'

## The Data

Details for the data can be found in the project [README.md](https://github.com/fazeelgm/UCB_ML_AI_Capstone/blob/main/README.md).

### Read the Data

In [11]:
# Which dataset to work from? Select sample size percentage

sample_file = data_utils.select_sample_csv_file(pct=10)
# sample_file = data_utils.select_sample_csv_file(pct=100)
# sample_file = data_utils.select_sample_csv_file(pct=75)
# sample_file = data_utils.select_sample_csv_file(pct=50)
# sample_file = data_utils.select_sample_csv_file(pct=25)

print(f'Selected sample file: {sample_file}')

Selected sample file: ../data/incidents_clean_10_pct.csv


In [12]:
current_raw_df, current_clean_df = data_utils.get_clean_data_from_csv(sample_file)

Reading file: ../data/incidents_clean_10_pct.csv ... Done: 89,458 rows, 37 columns
... Converting datetime to timeseries ... Done
... Setting index to datetime ... Done
Done


### Apply Feature Engineering Learnings

We will re-use the learnings from the Exploratory Data Analysis (EDA) and apply it to clean the data using two shared methods:

* `data_utils.preprocess_data()`
* `data_utils.fix_data_artifacts()`
* `data_utils.apply_synthetic_features()`

Please refer to the EDA notebook, 
[ExploratoryDataAnalysis.ipynb](https://github.com/fazeelgm/UCB_ML_AI_Capstone/blob/main/notebooks/ExploratoryDataAnalysis.ipynb), for details.

In [15]:
data = data_utils.preprocess_data(current_raw_df.copy())

Pre-processing ... 
... Dropping unwanted columns ... 
... preprocess_drop_cols: Column Unnamed: 0 dropped
... preprocess_drop_cols: Column esncag_-_boundary_file dropped
... preprocess_drop_cols: Column central_market/tenderloin_boundary_polygon_-_updated dropped
... preprocess_drop_cols: Column civic_center_harm_reduction_project_boundary dropped
... preprocess_drop_cols: Column hsoc_zones_as_of_2018-06-05 dropped
... preprocess_drop_cols: Column invest_in_neighborhoods_(iin)_areas dropped
... preprocess_drop_cols: Column report_type_code dropped
... preprocess_drop_cols: Column report_type_description dropped
... preprocess_drop_cols: Column filed_online dropped
... preprocess_drop_cols: Column intersection dropped
... preprocess_drop_cols: Column cnn dropped
... preprocess_drop_cols: Column point dropped
... preprocess_drop_cols: Column supervisor_district dropped
... preprocess_drop_cols: Column supervisor_district_2012 dropped
... preprocess_drop_cols: Column current_supervisor_d

In [16]:
# Fix data value artifacts that were discovered during EDA
data = data_utils.fix_data_artifacts(data)

Fixing data artifacts (in-place) ... 
... Category column:
    ..."Human Trafficking*"
    ..."Motor Vehicle Theft"
    ..."Weapons Offence"
Done


Create the new, synthetic features that were introduced during EDA:

In [18]:
data = data_utils.apply_synthetic_features(data)

Generating synthetic feature columns (in-place) ... 
... Adding columns ['hour', 'minute', 'day', 'month']'
... Adding column ['weekend']
... Adding column ['season']
... Adding column ['holiday']
... Adding column ['tod']
Done


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82888 entries, 2024-08-01 08:01:00 to 2018-10-02 16:53:00
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             82888 non-null  object 
 1   time             82888 non-null  object 
 2   year             82888 non-null  int64  
 3   dow              82888 non-null  object 
 4   category         82888 non-null  object 
 5   resolution       82888 non-null  object 
 6   police_district  82888 non-null  object 
 7   neighborhood     82888 non-null  object 
 8   latitude         82888 non-null  float64
 9   longitude        82888 non-null  float64
 10  hour             82888 non-null  int64  
 11  minute           82888 non-null  int64  
 12  day              82888 non-null  int64  
 13  month            82888 non-null  int64  
 14  weekend          82888 non-null  int64  
 15  season           82888 non-null  object 
 16  holiday          82888 

In [20]:
data.head(2)

,date,time,year,dow,category,resolution,police_district,neighborhood,latitude,longitude,hour,minute,day,month,weekend,season,holiday,tod
datetime,,,,,,,,,,,,,,,,,,
2024-08-01 08:01:00,2024/08/01,08:01,2024,Thursday,Other Miscellaneous,Open or Active,Mission,Mission,37.768272,-122.419983,8,1,1,8,0,Summer,False,Morning
2021-11-25 23:30:00,2021/11/25,23:30,2021,Thursday,Burglary,Open or Active,Northern,Haight Ashbury,37.773757,-122.432467,23,30,25,11,0,Fall,False,Evening


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82888 entries, 2024-08-01 08:01:00 to 2018-10-02 16:53:00
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             82888 non-null  object 
 1   time             82888 non-null  object 
 2   year             82888 non-null  int64  
 3   dow              82888 non-null  object 
 4   category         82888 non-null  object 
 5   resolution       82888 non-null  object 
 6   police_district  82888 non-null  object 
 7   neighborhood     82888 non-null  object 
 8   latitude         82888 non-null  float64
 9   longitude        82888 non-null  float64
 10  hour             82888 non-null  int64  
 11  minute           82888 non-null  int64  
 12  day              82888 non-null  int64  
 13  month            82888 non-null  int64  
 14  weekend          82888 non-null  int64  
 15  season           82888 non-null  object 
 16  holiday          82888 

In [22]:
# data.to_csv('../data/incidents_10.csv')

## Load Models

## SHAP

In [ ]:
import shap

In [ ]:
# X_test.info()

# X_test.iloc[0]

# start = time.time()

# explainer = shap.TreeExplainer(results_CV['RandomForestClassifier']['best_model'])
# print(f'Lapse time: {time_secs_to_msg(time.time() - start)}')

# shap_values = explainer.shap_values(X_test)
# print(f'Lapse time: {time_secs_to_msg(time.time() - start)}')

# explainer.expected_value[0]

# shap_values.shape, X_test.shape

# shap_values[0][0,:]

# shap_values.shape[1]

# shap.initjs()
# # shap.force_plot(exp.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:])
# shap.force_plot(exp.expected_value[0], shap_values[:, 0], X_test.iloc[0, :])

# print(f'Lapse time: {time_secs_to_msg(time.time() - start)}')